# Performance Optimization

## Setup

In [69]:
using Pkg

In [19]:
pkg"add BenchmarkTools"

 Resolving package versions...
  Updating `/opt/julia/environments/v1.3/Project.toml`
 [no changes]
  Updating `/opt/julia/environments/v1.3/Manifest.toml`
 [no changes]


In [20]:
pkg"add PyCall"

 Resolving package versions...
 Installed PyCall ───────── v1.91.2
 Installed MacroTools ───── v0.5.3
 Installed DataStructures ─ v0.17.9
  Updating `/opt/julia/environments/v1.3/Project.toml`
  [438e738f] + PyCall v1.91.2
  Updating `/opt/julia/environments/v1.3/Manifest.toml`
  [864edb3b] + DataStructures v0.17.9
  [1914dd2f] + MacroTools v0.5.3
  [438e738f] + PyCall v1.91.2
  Building PyCall → `/opt/julia/packages/PyCall/ttONZ/deps/build.log`


In [70]:
pkg"add Traceur"

  Updating registry at `/opt/julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %>                                 ]  15.2 % %                   ]  52.1 %===========================>             ]  67.3 %82.4 %7 %>]  99.9 % Resolving package versions...
 Installed Parsers ──── v0.3.11
 Installed Traceur ──── v0.3.0
 Installed MacroTools ─ v0.5.4
 Installed Cassette ─── v0.3.1
  Updating `/opt/julia/environments/v1.3/Project.toml`
  [37b6cedf] + Traceur v0.3.0
  Updating `/opt/julia/environments/v1.3/Manifest.toml`
  [7057c7e9] + Cassette v0.3.1
  [1914dd2f] ↑ MacroTools v0.5.3 ⇒ v0.5.4
  [69de0a69] ↑ Parsers v0.3.10 ⇒ v0.3.11
  [37b6cedf] + Traceur v0.3.0


In [71]:
pkg"update"

  Updating registry at `/opt/julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed IJulia ─ v1.21.0
  Updating `/opt/julia/environments/v1.3/Project.toml`
  [7073ff75] ↑ IJulia v1.20.2 ⇒ v1.21.0
  Updating `/opt/julia/environments/v1.3/Manifest.toml`
  [7073ff75] ↑ IJulia v1.20.2 ⇒ v1.21.0
  Building IJulia → `/opt/julia/packages/IJulia/fXyGm/deps/build.log`


In [72]:
pkg"precompile"

Precompiling project...
Precompiling IJulia


┌ Info: Precompiling IJulia [7073ff75-c697-5162-941a-fcdaad2a7d2a]
└ @ Base loading.jl:1273


Precompiling Traceur


┌ Info: Precompiling Traceur [37b6cedf-1f77-55f8-9503-c64b63398394]
└ @ Base loading.jl:1273


## Basis Example

The baseline for testing performance impact of different aspects is an implementation 
which is already quite effective - it is type stable and uses the fastest index order.

In [97]:
using BenchmarkTools

In [98]:
abstract type MyP end

In [99]:
struct MyParams{T} <: MyP
    a:: T
    b:: T
end

In [100]:
p = MyParams(1.5, 2.5)

MyParams{Float64}(1.5, 2.5)

In [101]:
n = 500;
N = rand(n, n);
M = rand(n, n);

Reduced test sample (for the slower implementations)

In [102]:
n2 = 50;
N2 = rand(n2, n2);
M2 = rand(n2, n2);

In [103]:
function my_mult_base(A:: AbstractArray{T,2}, B:: AbstractArray{T,2}, p:: MyP) where {T <: Number}
    @assert size(A, 2) == size(B, 1)
    C = zeros(T, size(A,1), size(B,2))
    for n=1:size(B, 2), k=1:size(A,2), m=1:size(A, 1)
        C[m,n] += p.a*A[m,k]*B[k,n]+p.b
    end
    C
end

my_mult_base (generic function with 1 method)

In [104]:
res_j = my_mult_base(N, M, p);

In [85]:
@btime res_j = my_mult_base($N, $M, $p);

  458.241 ms (2 allocations: 1.91 MiB)


In [86]:
res_j2 = my_mult_base(N2, M2, p);

In [87]:
@btime res_j2 = my_mult_base($N2, $M2, $p);

  473.758 μs (2 allocations: 19.64 KiB)


## 1. Type Stability

The base implementation is type stable. What happens if this would not be the case?
There are several ways to include type instability.

## Non-Constant Globals

In [88]:
function my_mult_global(A:: AbstractArray{T,2}, B:: AbstractArray{T,2}) where {T <: Number}
    @assert size(A, 2) == size(B, 1)
    C = zeros(T, size(A,1), size(B,2))
    for n=1:size(B, 2), k=1:size(A,2), m=1:size(A, 1)
        C[m,n] += p.a*A[m,k]*B[k,n]+p.b
    end
    C
end

my_mult_global (generic function with 1 method)

In [89]:
@assert my_mult_base(N2,M2,p) == my_mult_global(N2,M2)

In [90]:
@btime my_mult_global($N2,$M2);

  93.164 ms (1750002 allocations: 30.54 MiB)


In [91]:
@btime my_mult_base($N2,$M2,$p);

  473.549 μs (2 allocations: 19.64 KiB)


This is a slowdown by a factor of 200!

In [92]:
const pc = p
function my_mult_global_const(A:: AbstractArray{T,2}, B:: AbstractArray{T,2}) where {T <: Number}
    @assert size(A, 2) == size(B, 1)
    C = zeros(T, size(A,1), size(B,2))
    for n=1:size(B, 2), k=1:size(A,2), m=1:size(A, 1)
        C[m,n] += pc.a*A[m,k]*B[k,n]+pc.b
    end
    C
end

my_mult_global_const (generic function with 1 method)

In [93]:
@btime my_mult_global_const($N2,$M2);

  474.492 μs (2 allocations: 19.64 KiB)


Defining the global as constant removes the speed penalty.

__Conclusion__: Never use non-constant globals in function bodies!

## Structs with Non-Concrete Types

In [113]:
struct MyBadParams <: MyP
    a:: Number
    b:: AbstractFloat
end

In [114]:
p_bad = MyBadParams(1.5, 2.5)

MyBadParams(1.5, 2.5)

In [115]:
@btime my_mult_base($N2,$M2,$p_bad);

  51.494 ms (1000002 allocations: 15.28 MiB)


In [116]:
@btime my_mult_base($N2,$M2,$p_bad);

  52.812 ms (1000002 allocations: 15.28 MiB)


About a factor of 100 slower than the structure using parametric types.

__Conclusion__: never use abstract types in structures. Use concrete types (e.g. *Float64*)
or parametric types instead.

Note that abstract types are perfectly fine and give no performance penalty when used in functions.

## Type Instability inside Functions

In [25]:
function my_mult_instable(A:: AbstractArray{T,2}, B:: AbstractArray{T,2}, p:: MyP) where {T <: Number}
    @assert size(A, 2) == size(B, 1)
    C = zeros(T, size(A,1), size(B,2))
    for n=1:size(B, 2), k=1:size(A,2), m=1:size(A, 1)
        offset = size(A, 2) % 2 ==0 ? p.b : floor(Int64, p.b)
        C[m,n] += p.a*A[m,k]*B[k,n]+offset
    end
    C
end

my_mult_instable (generic function with 1 method)

In [26]:
@btime my_mult_instable($N2,$M2,$p);

  804.293 μs (2 allocations: 19.64 KiB)


In [27]:
@code_warntype my_mult_instable(N2,M2,p)

Variables
  #self#::Core.Compiler.Const(my_mult_instable, false)
  A::Array{Float64,2}
  B::Array{Float64,2}
  p::MyParams{Float64}
  C::Array{Float64,2}
  @_6::Union{Nothing, Tuple{Int64,Int64}}
  n@_7::Int64
  @_8::Union{Nothing, Tuple{Int64,Int64}}
  k@_9::Int64
  @_10::Union{Nothing, Tuple{Int64,Int64}}
  n@_11::Int64
  m::Int64
  offset::Union{Float64, Int64}
  k@_14::Int64
  @_15::Union{Float64, Int64}

Body::Array{Float64,2}
1 ──       Core.NewvarNode(:(C))
│          Core.NewvarNode(:(@_6))
│    %3  = Main.size(A, 2)::Int64
│    %4  = Main.size(B, 1)::Int64
│    %5  = (%3 == %4)::Bool
└───       goto #3 if not %5
2 ──       goto #4
3 ── %8  = Base.AssertionError("size(A, 2) == size(B, 1)")::AssertionError
└───       Base.throw(%8)
4 ┄─ %10 = $(Expr(:static_parameter, 1))::Core.Compiler.Const(Float64, false)
│    %11 = Main.size(A, 1)::Int64
│    %12 = Main.size(B, 2)::Int64
│          (C = Main.zeros(%10, %11, %12))
│    %14 = Main.size(B, 2)::Int64
│    %15 = (1:%14)::Core.Com

Not as bad as the other instabilities, but still a factor of 2 performance loss.

### Traceur

In [73]:
using Traceur

In [110]:
@code_warntype my_mult_base([1. 2.; 3. 4.],[5. 6.; 7. 8.],p)

Variables
  #self#::Core.Compiler.Const(my_mult_base, false)
  A::Array{Float64,2}
  B::Array{Float64,2}
  p::MyParams{Float64}
  C::Array{Float64,2}
  @_6::Union{Nothing, Tuple{Int64,Int64}}
  n@_7::Int64
  @_8::Union{Nothing, Tuple{Int64,Int64}}
  k@_9::Int64
  @_10::Union{Nothing, Tuple{Int64,Int64}}
  n@_11::Int64
  m::Int64
  k@_13::Int64

Body::Array{Float64,2}
1 ──       Core.NewvarNode(:(C))
│          Core.NewvarNode(:(@_6))
│    %3  = Main.size(A, 2)::Int64
│    %4  = Main.size(B, 1)::Int64
│    %5  = (%3 == %4)::Bool
└───       goto #3 if not %5
2 ──       goto #4
3 ── %8  = Base.AssertionError("size(A, 2) == size(B, 1)")::AssertionError
└───       Base.throw(%8)
4 ┄─ %10 = $(Expr(:static_parameter, 1))::Core.Compiler.Const(Float64, false)
│    %11 = Main.size(A, 1)::Int64
│    %12 = Main.size(B, 2)::Int64
│          (C = Main.zeros(%10, %11, %12))
│    %14 = Main.size(B, 2)::Int64
│    %15 = (1:%14)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, f

In [111]:
@trace my_mult_base([1. 2.; 3. 4.],[5. 6.; 7. 8.],p);

┌ Warning:  is assigned as Tuple{Int64,Int64}
└ @ abstractarray.jl:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ abstractarray.jl:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ abstractarray.jl:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ abstractarray.jl:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ array.jl:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ array.jl:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ In[103]:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ In[103]:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ In[103]:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ In[103]:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ In[103]:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ In[103]:-1


In [118]:
@code_warntype my_mult_base([1. 2.; 3. 4.],[5. 6.; 7. 8.],p_bad);

Variables
  #self#::Core.Compiler.Const(my_mult_base, false)
  A::Array{Float64,2}
  B::Array{Float64,2}
  p::MyBadParams
  C::Array{Float64,2}
  @_6::Union{Nothing, Tuple{Int64,Int64}}
  n@_7::Int64
  @_8::Union{Nothing, Tuple{Int64,Int64}}
  k@_9::Int64
  @_10::Union{Nothing, Tuple{Int64,Int64}}
  n@_11::Int64
  m::Int64
  k@_13::Int64

Body::Array{Float64,2}
1 ──       Core.NewvarNode(:(C))
│          Core.NewvarNode(:(@_6))
│    %3  = Main.size(A, 2)::Int64
│    %4  = Main.size(B, 1)::Int64
│    %5  = (%3 == %4)::Bool
└───       goto #3 if not %5
2 ──       goto #4
3 ── %8  = Base.AssertionError("size(A, 2) == size(B, 1)")::AssertionError
└───       Base.throw(%8)
4 ┄─ %10 = $(Expr(:static_parameter, 1))::Core.Compiler.Const(Float64, false)
│    %11 = Main.size(A, 1)::Int64
│    %12 = Main.size(B, 2)::Int64
│          (C = Main.zeros(%10, %11, %12))
│    %14 = Main.size(B, 2)::Int64
│    %15 = (1:%14)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false),

In [117]:
@trace my_mult_base([1. 2.; 3. 4.],[5. 6.; 7. 8.],p_bad);

┌ Warning:  is assigned as Tuple{Int64,Int64}
└ @ abstractarray.jl:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ abstractarray.jl:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ abstractarray.jl:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ abstractarray.jl:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ array.jl:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ array.jl:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ In[103]:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ In[103]:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ In[103]:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ In[103]:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ In[103]:-1
┌ Warning:  is assigned as Union{Nothing, Tuple{Int64,Int64}}
└ @ In[103]:-1
┌ Warning: dynamic dispatch to Base.getfield(p, a) * B

## 2. Memory Layout

## 3. Allocations

## 4. Removing Safety Belts

## 5. Parallelization

## Comparison to Python

In [9]:
using PyCall

In [10]:
np = pyimport("numpy")

PyObject <module 'numpy' from '/opt/conda/lib/python3.7/site-packages/numpy/__init__.py'>

In [11]:
py"""
import numpy as np
def my_mult_py(A, B, p):
    assert A.shape[1] == B.shape[0]
    C = np.zeros((A.shape[0], B.shape[1]))
    for m in range(A.shape[0]):
        for k in range(A.shape[1]):
            for n in range(B.shape[1]):
                C[m,n] += p.a*A[m,k]*B[k,n]+p.b
    return C
"""

In [33]:
res_py = py"my_mult_py($N2, $M2, $p)";

In [27]:
@btime res_py = py"my_mult_py($N2, $M2, $p)";

  1.206 s (2500050 allocations: 57.24 MiB)


In [37]:
@assert res_py ≈ res_j2

The pure Python implementation (with numpy only for data transfer) is ca 3000 times slower than Julia!

In [20]:
py"""
import numpy as np
def my_mult_np(A, B, p):
    C = p.a * A @ B + p.b*A.shape[1]
    return C
"""

In [57]:
res_np = py"my_mult_np($N2, $M2, $p)";

In [59]:
@assert res_np ≈ res_j2

In [58]:
@btime res_np = py"my_mult_np($N2, $M2, $p)";

  226.348 μs (70 allocations: 22.36 KiB)


In [60]:
@btime res_np = py"my_mult_np($N, $M, $p)";

  20.067 ms (71 allocations: 1.91 MiB)


Numpy is fast - ca 2 times the naive Julia implementation for small matrices and 20 times for large ones.
Note that numpy uses the highly optimed BLAS library for matrix multiplication.

## Comparison to Julia Builtin

In [64]:
my_func_b(A, B, p) = p.a .* (A*B) .+ p.b .* size(A,2)

my_func_b (generic function with 1 method)

In [65]:
@assert res_j ≈ my_func_b(N, M, p)

In [67]:
@btime my_func_b($N, $M, $p);

  13.822 ms (4 allocations: 3.81 MiB)


The Julia builtin matrix multiplication is even faster than numpy.